## Lectura de los datos

#### Librerias

In [1]:
import pandas as pd
import numpy as np
import keras_metrics
import tensorflow as tf
import shutil, os, sys
import matplotlib.pyplot as plt



from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers, layers
from tensorflow.keras import utils as k
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image_dataset_from_directory,image




#### Datos

In [2]:

file= 'metadata.csv'
dt=pd.read_csv(file, sep=",")

dt.head(1)

,patientid,offset,sex,age,finding,survival,intubated,intubation_present,went_icu,in_icu,...,date,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 28
0,2,0.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN


### Preprocesamiento

Vamos a ver en que proporcion encontramos las muestras de covid y las de otras enfermedades para que la red no se sobre entrene con un solo tipo de imagenes.

In [3]:
covid=[]
otro=[]

for i,r in dt.iterrows():
    if r['finding'] == 'COVID-19':
        covid.append(r)
    else:
        otro.append(r)

print("Co: ",len(covid) )
print("Otro: ",len(otro) )
print("Proporcion de Covid: ", len(covid)/(len(covid)+len(otro))*100)
    


Co:  296
Otro:  76
Proporcion de Covid:  79.56989247311827


Para que la funcion de keras: <i>image_dataset_from_directory</i> funcione para poder entrenar como queremos, separamos las fotos en dos capetas, una para covid y otra para el resto de enfermedades.

Como hemos visto hay muchas más radiografias de covid que de las otras enfermedades así que vamos a igualar el numero de muestras para cada clase.

##### Creamos los directorios

In [4]:
carpeta = os.getcwd()+os.sep+'images'


#SEPARACION EN LA CARPETA IMAGES
if not os.path.isdir(carpeta+os.sep+'covid') :
    os.mkdir(carpeta+os.sep+'covid')
#if not os.path.isdir(carpeta+os.sep+'covid'+os.sep+'covid'):
#    os.mkdir(carpeta+os.sep+'covid'+os.sep+'covid')
    
if not os.path.isdir(carpeta+os.sep+'otro') :
    os.mkdir(carpeta+os.sep+'otro')
#if not os.path.isdir(carpeta+os.sep+'otro'+os.sep+'otro'):
#    os.mkdir(carpeta+os.sep+'otro'+os.sep+'otro')
        
    
#SEPARACION EN LA CARPETA PARA EL TEST
if not os.path.isdir(os.getcwd()+os.sep+'test') :
    os.mkdir(os.getcwd()+os.sep+'test')
    if not os.path.isdir(os.getcwd()+os.sep+'test'+os.sep+'covid'):
        os.mkdir(os.getcwd()+os.sep+'test'+os.sep+'covid')
#    if not os.path.isdir(os.getcwd()+os.sep+'test'+os.sep+'covid'+os.sep+'covid'):
#        os.mkdir(os.getcwd()+os.sep+'test'+os.sep+'covid'+os.sep+'covid')
    if not os.path.isdir(os.getcwd()+os.sep+'test'+os.sep+'otro'):
        os.mkdir(os.getcwd()+os.sep+'test'+os.sep+'otro')
#    if not os.path.isdir(os.getcwd()+os.sep+'test'+os.sep+'otro'+os.sep+'otro'):
#        os.mkdir(os.getcwd()+os.sep+'test'+os.sep+'otro'+os.sep+'otro')

        
#SEPARACION EN LA CARPETA PARA LA VALIDACION
if not os.path.isdir(os.getcwd()+os.sep+'val') :
    os.mkdir(os.getcwd()+os.sep+'val')
    if not os.path.isdir(os.getcwd()+os.sep+'val'+os.sep+'covid'):
        os.mkdir(os.getcwd()+os.sep+'val'+os.sep+'covid')
#    if not os.path.isdir(os.getcwd()+os.sep+'val'+os.sep+'covid'+os.sep+'covid'):
#        os.mkdir(os.getcwd()+os.sep+'val'+os.sep+'covid'+os.sep+'covid')
    if not os.path.isdir(os.getcwd()+os.sep+'val'+os.sep+'otro'):
        os.mkdir(os.getcwd()+os.sep+'val'+os.sep+'otro')
#    if not os.path.isdir(os.getcwd()+os.sep+'val'+os.sep+'otro'+os.sep+'otro'):
#        os.mkdir(os.getcwd()+os.sep+'val'+os.sep+'otro'+os.sep+'otro')


#NUEVAS CARPETAS PARA ENTRENAMIENTO 
if not os.path.isdir(os.getcwd()+os.sep+'nuevas') :
    os.mkdir(os.getcwd()+os.sep+'nuevas')
    if not os.path.isdir(os.getcwd()+os.sep+'nuevas'+os.sep+'covid'):
        os.mkdir(os.getcwd()+os.sep+'nuevas'+os.sep+'covid')
        if not os.path.isdir(os.getcwd()+os.sep+'nuevas'+os.sep+'otro'):
            os.mkdir(os.getcwd()+os.sep+'nuevas'+os.sep+'otro')
            
            
#NUEVAS CARPETAS PARA TEST
if not os.path.isdir(os.getcwd()+os.sep+'nuevasT') :
    os.mkdir(os.getcwd()+os.sep+'nuevasT')
    if not os.path.isdir(os.getcwd()+os.sep+'nuevasT'+os.sep+'covid'):
        os.mkdir(os.getcwd()+os.sep+'nuevasT'+os.sep+'covid')
        if not os.path.isdir(os.getcwd()+os.sep+'nuevasT'+os.sep+'otro'):
            os.mkdir(os.getcwd()+os.sep+'nuevasT'+os.sep+'otro')
            
#NUEVAS CARPETAS PARA VALIDACION            
if not os.path.isdir(os.getcwd()+os.sep+'nuevasV') :
    os.mkdir(os.getcwd()+os.sep+'nuevasV')
    if not os.path.isdir(os.getcwd()+os.sep+'nuevasV'+os.sep+'covid'):
        os.mkdir(os.getcwd()+os.sep+'nuevasV'+os.sep+'covid')
        if not os.path.isdir(os.getcwd()+os.sep+'nuevasV'+os.sep+'otro'):
            os.mkdir(os.getcwd()+os.sep+'nuevasV'+os.sep+'otro')
            

In [5]:
dimensiones=[200,200]


##### Movemos las imagenes

In [6]:

foto=''
destino=''
ruta=carpeta+destino
noEstanco=[]
noEstanot=[]
NombresEntrenoCo=[]
NombresEntrenoOt=[]
NombresValCo=[]
NombresValOt=[]
NombresTestCo=[]
NombresTestOt=[]

numcovid=0  #flag para equilibrar las muestras
numValco=numTestco=0 #flag para coger muestras diferentes en test y prediccion covid
numValotro=numTestotro=0 #flag para coger muestras diferentes en test y `prediccion otra enfermedad
flag=0 #flag para no mover a las carpetas de images y que no aparezca en los dos directorios
pacienteAnteriorC=0
pacienteAnteriorO=0

fotosCo=0
fotosOt=0

for i,r in dt.iterrows():
    flag=0
    foto=r['filename']
    if r['finding'] == 'COVID-19': #si es covid
        if numcovid<(len(otro)-10):
            destino='covid'
            numcovid+=1
            
        elif numTestco < 6 and r['patientid'] != pacienteAnteriorC: #ejemplos de covid para test
            if os.path.exists(ruta+os.sep+foto):
                shutil.move(ruta+os.sep+foto,'test'+os.sep+'covid')
                NombresTestCo.append(r['filename'])
                numTestco+=1
                flag=1
                noEstanco.append(foto)
        elif numValco < 8:
            if os.path.exists(ruta+os.sep+foto):
                shutil.move(ruta+os.sep+foto,'val'+os.sep+'covid')
                NombresValCo.append(r['filename'])
                numValco+=1
    else: #Otra enfermedad
        if numTestotro < 6 and r['patientid'] != pacienteAnteriorO:#ejemplos de  otra enfermedad para prediccion
            if os.path.exists(ruta+os.sep+foto):
                shutil.move(ruta+os.sep+foto,'test'+os.sep+'otro')
                NombresTestOt.append(r['filename'])
                numTestotro+=1
                flag=1
                noEstanot.append(foto)
                
        elif numValotro < 8: #movemos 8 imagenes de otra enfermedad para validacion
            if os.path.exists(ruta+os.sep+foto):
                shutil.move(ruta+os.sep+foto,'val'+os.sep+'otro')
                NombresValOt.append(r['filename'])
                numValotro+=1
        destino='otro'
        
    if os.path.exists(ruta+os.sep+foto) and flag!=1:
        shutil.move(ruta+os.sep+foto,ruta+os.sep+destino)#+os.sep+destino
        #para aumentar los datos
        if destino=='covid':
            NombresEntrenoCo.append(r['filename'])
            fotosCo+=1
        else:
            fotosOt+=1
            NombresEntrenoOt.append(r['filename'])
    if r['finding'] == 'COVID-19':
        pacienteAnteriorC=r['patientid']
    else:
        pacienteAnteriorO=r['patientid']
    #print(ruta+os.sep+foto)
    

##### Aumentamos la caantidad de muestras que tenemos

In [107]:

aumento = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(dimensiones[0],dimensiones[1],3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)


nuevo=os.getcwd()+os.sep+'nuevas'
df=image.ImageDataGenerator(rotation_range=10, width_shift_range=0.08, height_shift_range=0.12,shear_range=0.12, zoom_range=0.08,channel_shift_range = 8, horizontal_flip=True)
#resul=image.DirectoryIterator(directory=carpeta, image_data_generator=df,target_size=(dimensiones[0],dimensiones[1]), color_mode='rgb', subset='training'
#                        class_mode='binary',batch_size=5)
#print(len(resul))
#imagen=df.flow_from_directory(carpeta, target_size=(dimensiones[0],dimensiones[1]), save_to_dir=nuevo, save_prefix='Nuevas', classes=['covid', 'otro'],class_mode="binary",batch_size=1, seed=1234)
#save_prefix='Nuevas',save_format='jpeg',


#ENTRENAMIENTO
Entreno=image.DirectoryIterator(directory=carpeta,image_data_generator=df,target_size=(dimensiones[0],dimensiones[1]), color_mode='rgb',
                         batch_size=35,class_mode='categorical')
#otr=image.DirectoryIterator(directory=carpeta+os.sep+'otro',image_data_generator=df,target_size=(dimensiones[0],dimensiones[1]), color_mode='rgb',
#                        class_mode=None, batch_size=20,save_format='jpeg')

#VALIDACION
Validacion=image.DirectoryIterator(directory='val',image_data_generator=df,target_size=(dimensiones[0],dimensiones[1]), color_mode='rgb',
                         batch_size=16,class_mode='categorical')
#otrVal=image.DirectoryIterator(directory='val'+os.sep+'otro',image_data_generator=df,target_size=(dimensiones[0],dimensiones[1]), color_mode='rgb',
#                        class_mode=None, batch_size=17,save_format='jpeg')

#TEST
Test=image.DirectoryIterator(directory='test',image_data_generator=df,target_size=(dimensiones[0],dimensiones[1]), color_mode='rgb',
                         batch_size=12,class_mode='categorical')
#Tot=image.DirectoryIterator(directory='test'+os.sep+'otro',image_data_generator=df,target_size=(dimensiones[0],dimensiones[1]), color_mode='rgb',
#                        class_mode=None, batch_size=10,save_format='jpeg')


#imagen.next()
#entreno=df.fit(imagen, rounds=4)


#l=[]
#for j in range(len(NombresEntrenoCo)):
#    for i in range(9):
#        #carpeta+os.sep+'covid'+os.sep+NombresEntrenoCo[j]
#        imagen=image.load_img(carpeta+os.sep+'covid'+os.sep+NombresEntrenoCo[j], grayscale=False, color_mode='rgb', target_size=(dimensiones[0],dimensiones[1]))
#        imagen=image.img_to_array(imagen)
#        imagen=np.array([imagen])
#        df.flow(imagen,batch_size=5, save_to_dir=os.getcwd()+os.sep+"jk", save_format='jpg',shuffle=True,save_prefix='Nuevas')
        #nueva = aumento(imagen)
        #cv2.imwrite(carpeta+os.sep+"covid",nueva[0].numpy().astype("uint8"))

#for j in range(len(NombresEntrenoOt)):
#    
#    imagen=image.load_img(carpeta+os.sep+'otro'+os.sep+'otro'+os.sep+NombresEntrenoCo[j], grayscale=False, color_mode='rgb', target_size=(dimensiones[0],dimensiones[1]))
#    imagen=image.img_to_array(imagen)
#    imagen=np.array(imagen)
#    imagen.next()
#    #image = np.expand_dims(ndimage.imread(carpeta+os.sep+'otro'+os.sep+'otro'+os.sep+NombresEntrenoCo[j]), 0)
#    df.fit(imagen)
#    for x, val in zip(df.flow(imagen,save_to_dir=os.getcwd()+os.sep+"nuevas"+os.sep+"covid",save_prefix='Nueva',save_format='jpeg'),range(5)):
#        pass
    #df.flow(imagen,batch_size=3, save_to_dir=carpeta+os.sep+'otro', save_format='jpg',shuffle=True)


#df.fit(image)

pd= layers.ZeroPadding2D(padding=2)

Found 323 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 12 images belonging to 2 classes.


In [7]:

for j in range(40):
    for i in range(len(cov)):
        cov.next()
    for i in range(len(otr)):
        otr.next()
        


NameError: name 'cov' is not defined

In [ ]:
data_list = []
batch_index = 0
clE=[]
for i in range(70):
    
    data = Entreno.next()
    data_list.append(data[0])
    clE.append(data[1])
        
    


In [ ]:
Entr=[data_list,clE]
len(data_list[0])

In [95]:
data_listV = []
batch_index = 0
clE2=[]

for i in range(50):
    while batch_index <= Validacion.batch_index:
        d = Validacion.next()
        data_listV.append(d[0])
        clE2.append(d[1])
        batch_index = batch_index + 1

##### Movemos las imagenes originales

In [ ]:
#ENTRENAMIENTO
for i in range(len(NombresEntrenoCo)):
    if os.path.exists(ruta+os.sep+'covid'+os.sep+'covid'+os.sep+NombresEntrenoCo[i]):
        shutil.move(ruta+os.sep+'covid'+os.sep+'covid'+os.sep+NombresEntrenoCo[i],'nuevas'+os.sep+'covid')
for i in range(len(NombresEntrenoOt)):
    if os.path.exists(ruta+os.sep+'otro'+os.sep+'otro'+os.sep+NombresEntrenoOt[i]):
        shutil.move(ruta+os.sep+'otro'+os.sep+'otro'+os.sep+NombresEntrenoOt[i],'nuevas'+os.sep+'otro')
#VALIDACION       
for i in range(len(NombresValCo)):
    if os.path.exists('val'+os.sep+'covid'+os.sep+'covid'+os.sep+NombresValCo[i]):
        shutil.move('val'+os.sep+'covid'+os.sep+'covid'+os.sep+NombresValCo[i],'nuevasV'+os.sep+'covid')
for i in range(len(NombresValOt)):
    if os.path.exists('val'+os.sep+'otro'+os.sep+'otro'+os.sep+NombresValOt[i]):
        shutil.move('val'+os.sep+'otro'+os.sep+'otro'+os.sep+NombresValOt[i],'nuevasV'+os.sep+'otro')


##### Guardamos las imagenes en memoria

In [ ]:

#entreno, val, test=image_dataset_from_directory(directory="images", image_size=(dimensiones[0],dimensiones[1]), subset="training", validation_split=0.2, seed=1234,
#                                      labels='inferred')



#entrenamiento
img_fit= image_dataset_from_directory(directory=nuevo, image_size=(dimensiones[0],dimensiones[1]), seed=764,
                                      labels='inferred',shuffle=True)


clases=img_fit.class_names

#validadcion
img_val=image_dataset_from_directory(directory='nuevasV', image_size=(dimensiones[0],dimensiones[1]), seed=9876,
                                      labels='inferred',shuffle=True)
#clases=img_val.class_names




#PARA MEJORAR EL RENDIMIENTO
AUTOTUNE = tf.data.AUTOTUNE
img_fit=img_fit.cache().prefetch(buffer_size=AUTOTUNE)
img_val=img_val.cache().prefetch(buffer_size=AUTOTUNE)

Normalizar= layers.experimental.preprocessing.Rescaling(1./255, input_shape=(dimensiones[0],dimensiones[1], 3))


### CREACION DEL MODELO

In [29]:
Normalizar= layers.experimental.preprocessing.Rescaling(1./255, input_shape=(dimensiones[0],dimensiones[1], 3))
pool= 5 #CON UN POOL MÁS GRANDE SE REDUCEN LOS PESOS
pd= layers.ZeroPadding2D(padding=2) #PADDING
model= Sequential([
    #aumento,
    Normalizar,
    pd,
    layers.Conv2D(filters=16,kernel_size=5,padding='same',activation='relu', strides=1, input_shape=(dimensiones[0],dimensiones[1], 3)),
    layers.MaxPooling2D(pool_size=pool),
    #layers.Dropout(0.05),
    layers.Conv2D(filters=32,kernel_size=5,padding='same',activation='relu', strides=1),
    layers.MaxPooling2D(pool_size=pool),
    #pd,
    #layers.Dropout(0.05),
    layers.Conv2D(filters=64,kernel_size=5,padding='same',activation='relu', strides=1),
    layers.MaxPooling2D(pool_size=pool),
    #pd,
    layers.Conv2D(filters=128,kernel_size=5,padding='same',activation='relu', strides=1),
    #pd,
    #layers.Conv2D(filters=256,kernel_size=5,padding='same',activation='relu', strides=1),
    #layers.MaxPooling2D(pool_size=pool),
    ##pd,
    #layers.Conv2D(filters=512,kernel_size=5,padding='same',activation='relu', strides=1),
    #layers.MaxPooling2D(pool_size=pool),
    layers.Dropout(0.5),
    layers.Flatten(),
    Dense(100,'relu'),
    #Dense(1000,'relu'),
    Dense(1,'sigmoid')

])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_1 (Rescaling)      (None, 200, 200, 3)       0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 204, 204, 3)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 204, 204, 16)      1216      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 40, 40, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 40, 40, 32)        12832     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 8, 8, 64)         

In [98]:
#print(Entr[1][0])
y_train=[]
for i in range(len(Entr[1][0])):
    #print(Entr[1][0][i][0])
    y_train.append(Entr[1][0][i][0])
y_train=np.array(y_train)
y_test=[]   
for i in range(len(Vali[1][0])):
    #print(Entr[1][0][i][0])
    y_test.append(Vali[1][0][i][0])
y_test=np.array(y_test)

len(Entr[1][0])

323

### ENTRENAMIENTO

In [97]:
opti= optimizers.Adam(learning_rate=0.2)#learning_rate=0.05,epsilon=0.8
vueltas=10
#Entrenamiento=np.array(Entreno)
#Valida=np.array(Validacion)

#y_train = np.asarray(Entr[1]).astype('float32').reshape((-1,1))
#y_test = np.asarray(Vali[1]).astype('float32').reshape((-1,1))

model.compile(optimizer=opti, loss='binary_crossentropy', metrics=['accuracy'])
historico=model.fit(x=Entr[0],y=y_train, validation_data=(Vali[0],y_test) ,epochs= vueltas,shuffle=True)

Epoch 1/10
11/11 [==============================] - 5s 413ms/step - loss: 0.6691 - accuracy: 0.6232 - val_loss: 0.7869 - val_accuracy: 0.5000
Epoch 2/10
11/11 [==============================] - 5s 419ms/step - loss: 0.6699 - accuracy: 0.6399 - val_loss: 0.7510 - val_accuracy: 0.5000
Epoch 3/10
11/11 [==============================] - 5s 430ms/step - loss: 0.6690 - accuracy: 0.6194 - val_loss: 0.7200 - val_accuracy: 0.5000
Epoch 4/10
11/11 [==============================] - 4s 398ms/step - loss: 0.6509 - accuracy: 0.6502 - val_loss: 0.7161 - val_accuracy: 0.5000
Epoch 5/10
11/11 [==============================] - 4s 390ms/step - loss: 0.6654 - accuracy: 0.6180 - val_loss: 0.7431 - val_accuracy: 0.5000
Epoch 6/10
11/11 [==============================] - 4s 400ms/step - loss: 0.6865 - accuracy: 0.5910 - val_loss: 0.7617 - val_accuracy: 0.5000
Epoch 7/10
11/11 [==============================] - 5s 402ms/step - loss: 0.6504 - accuracy: 0.6531 - val_loss: 0.7446 - val_accuracy: 0.5000
Epoch 

##### RESULTADOS DEL ENTRENAMIENTO SIN DATA AUGMENTATION

In [ ]:


acc = historico.history['accuracy']
val_acc = historico.history['val_accuracy']

loss = historico.history['loss']
val_loss = historico.history['val_loss']

epochs_range = range(vueltas)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
#Con los datos aumentados

#historicoNu=model.fit(resul, validation_data= img_val, epochs=vueltas)

##### RESULTADOS DEL ENTRENAMIENTO CON LAS NUEVAS IMAGENES

In [ ]:

for j in range(10):
    for i in range(len(Test)):
        Test.next()
    
        


In [ ]:
#TEST
for i in range(len(NombresTestCo)):
    if os.path.exists('test'+os.sep+'covid'+os.sep+'covid'+os.sep+NombresTestCo[i]):
        shutil.move('test'+os.sep+'covid'+os.sep+'covid'+os.sep+NombresTestCo[i],'nuevasT'+os.sep+'covid')
for i in range(len(NombresTestOt)):
    if os.path.exists('test'+os.sep+'otro'+os.sep+'otro'+os.sep+NombresTestOt[i]):
        shutil.move('test'+os.sep+'otro'+os.sep+'otro'+os.sep+NombresTestOt[i],'nuevasT'+os.sep+'otro')
        
img=image_dataset_from_directory(directory="nuevasT", image_size=(dimensiones[0],dimensiones[1]), 
                                      labels='inferred')

In [ ]:





tf.nn.sigmoid(model.predict(Test))

In [ ]:
test_loss, test_acc = model.evaluate(Test)
